# Training model on cartography classes

I want to see whether selecting easy-to-learn, hard-to-learn or ambiguous data instances will omprove performance on our 2k curriculum as well.
Might make multiple curriculum with:
- an equal mix of the three groups
- just ambiguous 
- just hard-to-learn

Paper: Swayamdipta, et all. (2020). Dataset cartography: Mapping and diagnosing datasets with training dynamics. arXiv preprint arXiv:2009.10795.

## Importing the datasets
### Importing cartography data

In [ ]:

import pandas as pd

data = pd.read_csv('snli_roberta_0_6_data_map_coordinates.jsonl.txt', header=None)
data.columns = ["guid", "index", "confidence", "variability", "correctness"]


In [13]:
print(len(data))
print(data) # 4804607632

549367
                         guid         index               confidence  \
0       {"guid":4446013212032       index:0  confidence:0.6756780396   
1        {"guid":568940932410       index:1  confidence:0.8437704568   
2       {"guid":4107496324431       index:2  confidence:0.8646389991   
3        {"guid":151853983012       index:3  confidence:0.8720001976   
4       {"guid":6041167176312       index:4   confidence:0.653920905   
...                       ...           ...                      ...   
549362  {"guid":5732659631411  index:549362  confidence:0.9975628853   
549363  {"guid":3920105265011  index:549363  confidence:0.9949818949   
549364  {"guid":6888801884010  index:549364  confidence:0.9364117483   
549365   {"guid":189022647122  index:549365  confidence:0.7770978212   
549366  {"guid":4506417051031  index:549366  confidence:0.9137195448   

                     variability     correctness  
0        variability:0.301201277  correctness:4}  
1       variability:0.3148

### Importing SNLI data

In [27]:
# if assigntools not yet downloaded run line
# ! git clone https://github.com/kovvalsky/assigntools.git

# if zip file of SNLI data not yet downloaded run lines
# !wget https://nlp.stanford.edu/projects/snli/snli_1.0.zip
# !unzip snli_1.0.zip

Cloning into 'assigntools'...


In [30]:
from assigntools.LoLa.read_nli import snli_jsonl2dict, sen2anno_from_nli_problems
from assigntools.LoLa.sen_analysis import spacy_process_sen2tok, display_doc_dep

In [ ]:
# this is Lasha's code for downloading SNLI
SNLI, S2A = snli_jsonl2dict('snli_1.0') 

Found .json files for ['dev', 'test', 'train'] parts
processing DEV:	

10000it [00:02, 4381.01it/s]


9842 problems read
0 problems have a wrong annotator label
processing TEST:	

10000it [00:01, 8566.13it/s]


9824 problems read
0 problems have a wrong annotator label
processing TRAIN:	

550152it [01:13, 7473.49it/s] 

549169 problems read
198 problems have a wrong annotator label
Most common weird labels: //(198)


In [31]:
data_snli = pd.read_json('snli_1.0_train.jsonl', lines=True)
# data.columns = ["guid", "index", "confidence", "variability", "correctness"]
# print(data_snli["annotator_labels"])

0               [neutral]
1         [contradiction]
2            [entailment]
3               [neutral]
4            [entailment]
               ...       
550147    [contradiction]
550148          [neutral]
550149          [neutral]
550150    [contradiction]
550151       [entailment]
Name: annotator_labels, Length: 550152, dtype: object


In [26]:
print(data_snli.shape)

(550152, 10)


Have to find a way to match up the number of instances of the cartography data to the number of instances in the original SNLI dataset in order to match the ID's from the cartography values to the sentences to train the model on.
- The cartography dataset has *549367*
- The original SNLI dataset downloaded from Stanford or huggingface has *550152*
- From Lasha's code has *549169* with 198 with wrong annotater labels

This suggests that if we delete the instances with wrong annotated labels from the cartography dataset that the instances from the cartography dataset would match that of Lasha's processed dataset and we could match up the ID's of the two datasets.

## Making the curiculi
I want to use the different groups (hard-to-learn, easy-to-learn, ambiguous) based on the values in the cartography dataset. Then try the same training sets (just hard-to-learn and just ambiguous) as used in the paper and, if times allows, also a mix. This will allow us to answer the question can we find the same patterns as the paper in our curriculums of 2k? 

Tasks:
- Look into what the values are that they used in the paper to destinguish the three groups. Paper doesn't mention values --> look at code
- How to take 2k datapoints from those bins. Do you want to take those randomly or make smaller bins inside of the groups to select a good sample of that particular group?
